In [ ]:
import panel as pn
from bquant_app.app import DNApp

pn.extension("tabulator")
app = DNApp()
app.cb_ticker.value = "DE000A4DFHL5 Corp"  # sous-jacent laissé vide (dérivation auto)
app.view()




import pandas as pd
import hvplot.pandas  # noqa
from bquant_app.bql_fetch import fetch_timeseries_with_bql, compute_nuke_series_with_bql
from bquant_app.logic import compute_dollar_neutral

cb = "DE000A4DFHL5 Corp"
start, end = "2024-01-01", pd.Timestamp.today().date().isoformat()

ts = fetch_timeseries_with_bql(cb_ticker=cb, udly_ticker=None, start=start, end=end, freq="BUSINESS_DAYS")

df_delta = compute_dollar_neutral(cb_close=ts.cb_close, udly_close=ts.udly_close, ud_delta=ts.ud_delta,
                                  anchor_date=None, method="delta")
anchor = df_delta.index[0]
CB0, U0 = df_delta.at[anchor, "cb_close"], df_delta.at[anchor, "udly_close"]
nuke_series = compute_nuke_series_with_bql(cb_ticker=cb, udly_close=df_delta["udly_close"],
                                           anchor_cb_price=float(CB0), anchor_udly_price=float(U0))
df_bql = compute_dollar_neutral(cb_close=ts.cb_close, udly_close=ts.udly_close, ud_delta=ts.ud_delta,
                                anchor_date=anchor, method="external_nuke", nuke_series=nuke_series)

overlay = df_delta["dn"].rename("Delta (linéaire)")\
    .to_frame().join(df_bql["dn"].rename("BQL nuke")).hvplot(height=350, line_width=2, title="DN DE000A4DFHL5 Corp")
overlay
